# Analysis of intraday storage

Using the base case hydrology and P&E modelling, determine the maximum volume of water that can be held back during two offpeak periods and then released during morning and evening peak hour periods. 

Calculate the energy transferred from the storage (off-peak) to peak periods.

Calculate the rise in the headpond and tailrace during each storage period (and therefore fall during each peak period). 

### Setup

In [233]:
import numpy as np
import pandas as pd
import datetime
import math

In [234]:
def tailwaterLevel(flow):
    if fixed_tailwater_level>0:
        return fixed_tailwater_level
    else:
        log_flow=math.log10(flow)
        if log_flow>=3.8:
            return -21.429*log_flow**2+182.03*log_flow+597.15   
        elif log_flow>=2.3:
            return 6.5299*log_flow**2-30.155*log_flow+999.52   
        else:
            return -2.6828*log_flow**2+11.265*log_flow+952.68

In [413]:
flow_data='../output_data/ngonye_flow_monthly.csv'
energy_data='../output_data/base_pe_monthly.csv'
dam_data='dam_calmonthly.csv'

canal_capacity=1100
rated_flow_unit=250

#offpeak_min_flow=rated_flow_unit
#offpeak_min_flow=rated_flow_unit * 0.5
#offpeak_min_flow=50


storage_curve=10 # 10 Mm3/m headpond rise - From Feasibility Study Rev D Appendix H - Mott MacDonald
max_power=185 # Max output for whole plant
#max_power=205
fixed_tailwater_level = 0

night_hours = 10 # Night off-peak
morning_hours = 3 # Length of morning peak
standard_hours = 9 # Time between morning peak and evening peak when storing can occur
evening_hours = 2 # Length of evening peak

nostore=False

### Load data

In [414]:
dam = pd.read_csv(dam_data).set_index('month')
dam.columns

Index(['Unnamed: 0', 'day_of_week', 'value_mean', 'price_weight_mean',
       'price_max', 'price_min', 'price_mean', 'power_max', 'power_min',
       'power_mean', 'price_morning_max', 'price_evening_max',
       'price_standard', 'off_price_mean', 'standard_price_mean',
       'morning_price_mean', 'evening_price_mean', 'off_hour_mean',
       'standard_hour_mean', 'morning_hour_mean', 'evening_hour_mean',
       'off_value', 'standard_value', 'morning_value', 'evening_value',
       'volume', 'off_volume', 'standard_volume', 'morning_volume',
       'evening_volume', 'value'],
      dtype='object')

In [415]:
pe = pd.read_csv(energy_data).set_index('MonthId')
pe.columns

Index(['Flow', 'Exceedance', 'Year', 'Month', 'WaterYear', 'WaterMonth',
       'Flow_difference', 'Flow_difference_pct', 'EWRChannelA', 'EWRChannelC',
       'EWRChannelD', 'EWRChannelE', 'EWRChannelFG', 'EWRTotal',
       'EWRProportion', 'FlowAvailableForGeneration', 'FlowCanal',
       'LowFlowShutoff', 'Turbines', 'FlowTurbine1', 'FlowTurbine2',
       'FlowTurbine3', 'FlowTurbine4', 'FlowSpill', 'SpillChannelA',
       'SpillChannelC', 'SpillChannelD', 'SpillChannelE', 'SpillChannelFG',
       'FlowChannelA', 'FlowChannelC', 'FlowChannelD', 'FlowChannelE',
       'FlowChannelFG', 'FlowLeftChannel', 'LevelTailwater', 'LevelHeadpond',
       'HeadlossLeftChannel', 'HeadlossCanal', 'LevelForebay',
       'HeadlossTurbine1', 'HeadlossTurbine2', 'HeadlossTurbine3',
       'HeadlossTurbine4', 'HeadTurbine1', 'HeadTurbine2', 'HeadTurbine3',
       'HeadTurbine4', 'LowHeadShutoff', 'LoadFactorTurbine1',
       'LoadFactorTurbine2', 'LoadFactorTurbine3', 'LoadFactorTurbine4',
       'EffT

In [416]:
effective_efficiency = 0.9211999999999999

In [417]:
tmp=pe[['Year','Month','WaterYear','WaterMonth','Days','LowHeadShutoff','Flow','EWRTotal','FlowSpill','FlowAvailableForGeneration','FlowCanal','Power','Energy','LevelHeadpond','LevelTailwater', 'HeadTurbine1']].copy()

tmp['LowHeadShutoff'] = tmp['LowHeadShutoff'].fillna(0)
work=tmp
work.columns

Index(['Year', 'Month', 'WaterYear', 'WaterMonth', 'Days', 'LowHeadShutoff',
       'Flow', 'EWRTotal', 'FlowSpill', 'FlowAvailableForGeneration',
       'FlowCanal', 'Power', 'Energy', 'LevelHeadpond', 'LevelTailwater',
       'HeadTurbine1'],
      dtype='object')

### Peak Periods Requirements for maximum output

In [418]:
#Total generation volume Mm3
work['GenerationVol'] = work['FlowCanal'] * (24 * 60 * 60) / 1e+06

#Rough calculation of the maximum flow during peak time - flow that produced maximum output at current head and high efficiency
work['MaxGenerationFlow'] = max_power / work['HeadTurbine1'] / 9.81 / effective_efficiency * 1000
#Limit maximum flow to canal capacity
work['MaxGenerationFlow'] = np.minimum(work['MaxGenerationFlow'],canal_capacity)

#Maximum flow that could be used by the peaks from storage m3/s
work['MaxFlowFromStore'] = work['MaxGenerationFlow'] - work['FlowCanal']

#Max volume that could be used by the peaks Mm3
work['MorningMaxVolFromStore'] = work['MaxFlowFromStore'] * (morning_hours * 60 * 60) / 1e+06
work['EveningMaxVolFromStore'] = work['MaxFlowFromStore'] * (evening_hours * 60 * 60) / 1e+06


### Max volume which the off-peak (night) and standard (day) periods can store

In [419]:
if (nostore):
    #Max volume that could be taken from the night period for storage Mm3
    work['NightMaxVolToStore'] = 0

    #Max volume that could be taken from the standard period for storage Mm3
    work['StandardMaxVolToStore'] = 0

else:  
    #Max volume that could be taken from the night period for storage Mm3
    work['NightMaxVolToStore'] = ((work['FlowAvailableForGeneration'] - offpeak_min_flow) * (night_hours * 60 * 60) / 1e+06).clip(lower=0)

    #Max volume that could be taken from the standard period for storage Mm3
    work['StandardMaxVolToStore'] = ((work['FlowAvailableForGeneration'] - offpeak_min_flow) * (standard_hours * 60 * 60) / 1e+06).clip(lower=0)

    

### Storage volumes

In [420]:
#Volume per night that will be stored (min of available or needed)
work['NightStoreVol'] = work[['MorningMaxVolFromStore','NightMaxVolToStore']].min(axis=1)

#Not enough head for generation so no point storing
work.loc[work['LowHeadShutoff'] == 1.0, 'NightStoreVol'] = 0

#Volume per standard period that will be stored (min of available or needed)
work['StandardStoreVol'] = work[['EveningMaxVolFromStore','StandardMaxVolToStore']].min(axis=1)

#Not enough head for generation so no point storing
work.loc[work['LowHeadShutoff'] == 1.0, 'NightStoreVol'] = 0


### Flows

In [421]:
#Night
work['NightExtraFlow']=(work['NightStoreVol'] / (60 * 60 * night_hours / 1e+06)) # Flow diverted to storage during night
work['NightFlowCanal']=work['FlowCanal'] - work['NightExtraFlow'] #Resulting canal flow during off-peak

#Morning
work['MorningExtraFlow']=(work['NightStoreVol'] / (60 * 60 * morning_hours / 1e+06)) # Extra flow from storage during morning peak 
work['MorningFlowCanal']=work['FlowCanal'] + work['MorningExtraFlow'] #Resulting canal flow during morning peak

#Standard
work['StandardExtraFlow']=(work['StandardStoreVol'] / (60 * 60 * standard_hours / 1e+06)) # Flow diverted to storage during standard period
work['StandardFlowCanal']=work['FlowCanal'] - work['StandardExtraFlow'] #Resulting canal flow during standard period

#Evening
work['EveningExtraFlow']=(work['StandardStoreVol'] / (60 * 60 * evening_hours / 1e+06)) # Extra flow from storage during evening peak 
work['EveningFlowCanal']=work['FlowCanal'] + work['EveningExtraFlow'] #Resulting canal flow during evening peak

### Total flow

In [422]:
work['NightTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['NightFlowCanal']
work['MorningTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['MorningFlowCanal']
work['StandardTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['StandardFlowCanal']
work['EveningTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['EveningFlowCanal']

### Total volumes

In [423]:
work['NightVol']=work['NightFlowCanal'] * night_hours * 60 * 60 / 1e+06
work['MorningVol']=work['MorningFlowCanal'] * morning_hours * 60 * 60 / 1e+06
work['StandardVol']=work['StandardFlowCanal'] * standard_hours * 60 * 60 / 1e+06
work['EveningVol']=work['EveningFlowCanal'] * evening_hours * 60 * 60 / 1e+06

### Energy

In [424]:
work['NightEnergy'] = work['Energy'] / work['GenerationVol'] * work['NightVol']
work['MorningEnergy'] = work['Energy'] / work['GenerationVol'] * work['MorningVol']
work['StandardEnergy'] = work['Energy'] / work['GenerationVol'] * work['StandardVol']
work['EveningEnergy'] = work['Energy'] / work['GenerationVol'] * work['EveningVol']

work['MorningEnergyTransfer'] = work['Energy'] / work['GenerationVol'] * work['NightStoreVol']
work['EveningEnergyTransfer'] = work['Energy'] / work['GenerationVol'] * work['StandardStoreVol']


### Power

In [425]:
work['NightPower'] = work['NightEnergy'] / night_hours
work['MorningPower'] = work['MorningEnergy'] / morning_hours
work['StandardPower'] = work['StandardEnergy'] / standard_hours
work['EveningPower'] = work['EveningEnergy'] / evening_hours

### Headpond levels

In [426]:
work['NightHeadpondChange']=work['NightStoreVol'] / storage_curve
work['StandardHeadpondChange']=work['StandardStoreVol'] / storage_curve

### Tailwater levels

In [427]:
work['NightTailwaterLevel']=work['NightTotalFlow'].apply(tailwaterLevel) 
work['MorningTailwaterLevel']=work['MorningTotalFlow'].apply(tailwaterLevel) 
work['StandardTailwaterLevel']=work['StandardTotalFlow'].apply(tailwaterLevel) 
work['EveningTailwaterLevel']=work['EveningTotalFlow'].apply(tailwaterLevel) 

work['NightTailwaterChange']=work['MorningTailwaterLevel'] - work['NightTailwaterLevel']
work['StandardTailwaterChange']=work['EveningTailwaterLevel'] - work['StandardTailwaterLevel']



In [428]:

work.loc[work['MorningMaxVolFromStore'] < work['NightMaxVolToStore'],'NightSupplyLimit'] = 'False'
work.loc[work['MorningMaxVolFromStore'] > work['NightMaxVolToStore'],'NightSupplyLimit'] = 'True'
work.loc[work['EveningMaxVolFromStore'] < work['StandardMaxVolToStore'],'StandardSupplyLimit'] = 'False'
work.loc[work['EveningMaxVolFromStore'] > work['StandardMaxVolToStore'],'StandardSupplyLimit'] = 'True'

work.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1116 entries, 1924.1 to 2017.09
Data columns (total 61 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year                        1116 non-null   float64
 1   Month                       1116 non-null   float64
 2   WaterYear                   1116 non-null   float64
 3   WaterMonth                  1116 non-null   float64
 4   Days                        1116 non-null   int64  
 5   LowHeadShutoff              1116 non-null   float64
 6   Flow                        1116 non-null   float64
 7   EWRTotal                    1116 non-null   float64
 8   FlowSpill                   1116 non-null   float64
 9   FlowAvailableForGeneration  1116 non-null   float64
 10  FlowCanal                   1116 non-null   float64
 11  Power                       1116 non-null   float64
 12  Energy                      1116 non-null   float64
 13  LevelHeadpond          

In [429]:
work['NightValue']=work['NightEnergy']*work.join(dam, on='Month')['off_price_mean']/1000000
work['MorningValue']=work['MorningEnergy']*work.join(dam, on='Month')['morning_price_mean']/1000000
work['StandardValue']=work['StandardEnergy']*work.join(dam, on='Month')['standard_price_mean']/1000000
work['EveningValue']=work['EveningEnergy']*work.join(dam, on='Month')['evening_price_mean']/1000000

work['Value']=work['NightValue'] + work['MorningValue'] + work['StandardValue'] + work['EveningValue']
work['Price']=work['Value']/work['Energy']*1000000

work[['Value','Energy','Price']]

,Value,Energy,Price
MonthId,,,
1924.10,0.402405,6668.218097,60.346717
1924.11,1.041438,15948.234986,65.301150
1924.12,2.784171,35452.577303,78.532252
1925.01,5.567236,87660.499151,63.509055
1925.02,7.609005,122211.453841,62.260984
...,...,...,...
2017.05,7.266267,119278.435042,60.918527
2017.06,8.847277,112359.569599,78.740748
2017.07,5.208842,46774.689331,111.360277


In [430]:
calmonths=work.reset_index()[[
    'Month','Energy','Value','Price',
    'NightEnergy','MorningEnergy','StandardEnergy','EveningEnergy',
    'NightValue','MorningValue','StandardValue','EveningValue',
    'MorningEnergyTransfer','EveningEnergyTransfer']].groupby('Month').mean()

calmonths

,Energy,Value,Price,NightEnergy,MorningEnergy,StandardEnergy,EveningEnergy,NightValue,MorningValue,StandardValue,EveningValue,MorningEnergyTransfer,EveningEnergyTransfer
Month,,,,,,,,,,,,,
1.0,83903.037082,5.376567,64.759688,28508.631826,16938.846594,27142.264641,11313.294022,1.013798,1.298689,1.963105,1.100976,6450.966958,4321.374265
2.0,98181.956051,6.269525,64.439596,38350.909677,14830.983184,35054.812881,9945.250309,1.363589,1.197828,2.656607,1.051501,2558.238677,1763.420638
3.0,103752.601623,5.995014,58.462967,42954.818683,13855.369434,38498.346326,9571.812850,1.375201,1.065148,2.598334,0.956332,745.326022,831.783909
4.0,98323.751796,5.740601,58.290614,41243.019758,12594.578033,37002.699669,8552.190769,1.294673,0.981102,2.566312,0.898515,170.517004,269.483417
5.0,114093.640328,6.988330,61.332286,47068.414113,14732.307732,42437.188440,9855.730044,1.543243,1.237968,3.096995,1.110125,470.602691,347.926683
6.0,105046.469668,8.359584,80.920859,40949.429773,15950.741297,37463.272888,10683.025710,1.632885,1.941499,3.295975,1.489226,2819.932589,1929.153238
7.0,60456.620121,6.014047,101.695399,16817.710050,15929.625849,16842.663997,10866.620226,0.725774,2.107417,1.588167,1.592689,8372.548334,5828.568549
8.0,36940.203080,3.827327,102.754316,9043.496763,10965.779906,8641.894483,8289.031929,0.365540,1.424256,0.798495,1.239036,6348.254521,5210.681672
9.0,26741.073843,2.163719,79.146492,8145.621120,6339.127212,7426.115780,4830.209732,0.301044,0.621031,0.584080,0.657565,2996.492981,2601.786912


In [431]:
#months.to_csv('../intraday/intraday_monthly_nostore.csv')
#months.to_csv('../intraday/intraday_monthly_rated.csv')
#months.to_csv('../intraday/intraday_monthly_halfrated.csv')
months.to_csv('../intraday/intraday_monthly_50cumec.csv')
#months.to_csv('../intraday/intraday_monthly_50cumec_205mw.csv')

#calmonths.to_csv('../intraday/intraday_calmonthly_nostore.csv')
#calmonths.to_csv('../intraday/intraday_calmonthly_rated.csv')
#calmonths.to_csv('../intraday/intraday_calmonthly_halfrated.csv')
calmonths.to_csv('../intraday/intraday_calmonthly_50cumec.csv')
#months.to_csv('../intraday/intraday_calmonthly_50cumec_205mw.csv')